# Conception BDD Normalisée et Conforme au RGPD (SIDORA AI)
---
## 

## CRUD pour verifier les donneés

In [ ]:
# import
from datetime import datetime
import pandas as pd
from passlib.hash import argon2

from sqlalchemy import Column, Integer, String, Boolean, DateTime, ForeignKey, create_engine
from sqlalchemy.orm import relationship, declarative_base, sessionmaker
from sqlalchemy.sql import func



In [ ]:
# db_reader.py

engine = create_engine("sqlite:///BD_Ventes_de_jeux_video.bd") 

# session
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
# Function pour creater un Client, Commande
def create_Client(age_id, region_id):
    try:
        session.add(Client(
        age_id = age_id,
        region_id = region_id,
        ))
        session.commit()
    except Exception as e:
        session.rollback()
        raise e
    
def create_donne_pers(login, mot_de_passe_hash):
    try:
        session.add(DonnePersonnel(
            login = login,
            mot_de_passe_hash = mot_de_passe_hash
            date_suppression = None,
            anonymise = False,
            ))
        session.commit()
    except Exception as e:
        session.rollback()
        raise e

def create_commande(client_id, produit_id, nb_produit):
    try:
        session.add(Commande(
            client_id = client_id,
            produit_id = produit_id,
            nb_produit = nb_produit,
            ))
        session.commit()
    except Exception as e:
        session.rollback()
        raise e

In [ ]:
# Function pour faire un roquet et pour retourner DataFrame
def read_table(table_class, limit=None, filter_exp=None):
    """
    table_class: class SQLAlchemy (par exemple.: Client, Commande)
    filter: opt
    limit: opt

    """
    query = session.query(table_class)
    
    if filter_exp is not None:
        query = query.filter(filter_exp)
    
    if limit:
        query = query.limit(limit)

    if query and (table_class is Client):
        session.query(Client).update(
            {Client.date_derniere_utilisation: func.now()},
            synchronize_session=False
        )
        session.commit()
    
    df = pd.read_sql(query.statement, session.bind)
    return df

In [51]:

df = read_table(Client, filter_exp=(Client.client_id < 5))
df

,client_id,age_id,region_id,date_creation,date_derniere_utilisation
0,1,1,0,2025-11-30 18:30:06,2025-11-30 18:46:55
1,2,0,2,2025-11-30 18:30:06,2025-11-30 18:46:55
2,3,2,1,2025-11-30 18:30:06,2025-11-30 18:46:55
3,4,0,0,2025-11-30 18:30:06,2025-11-30 18:46:55


In [29]:
session.close()
engine.dispose()